# SlideRule Voila Demo

SlideRule is an on-demand science data processing service that runs in the cloud and responds to REST API calls to process and return science results.

This web page demonstrates a simple use of SlideRule to return elevations within a small region of interest using processing parameters specified at the time of the request.  The web page is implemented inside a Jupyter Notebook using SlideRule's Python client, and is statically served using Voila.  The source code for what you see here can be found at: https://github.com/ICESat2-SlideRule/sliderule-python/blob/main/examples/voila_demo.ipynb. 

For more information on SlideRule, how to install it and use it for your own analysis applications, please see our website at [icesat2sliderule.org](http://icesat2sliderule.org).

In [ ]:
# load the necessary packages
from sliderule import icesat2, ipysliderule, io
import ipywidgets as widgets
import geopandas
import logging
import warnings
import time

In [ ]:
# turn off warnings for demo
warnings.filterwarnings('ignore')

In [ ]:
# create global variables
url_textbox = None
atl06_rsps = None
parms = None
results = []
SRwidgets = ipysliderule.widgets()

### Step 1: Select Region of Interest

Use the map to draw a polygon or bounding box around your region of interest.

In [ ]:
# create ipyleaflet map in specified projection
m = ipysliderule.leaflet('Global')
m.map

### Step 2: Choose Processing Parameters and Run SlideRule

Select the processing parameters you want to use to calculate gridded elevations for the region of interest you created above.  After you are done, click the "Run SlideRule!" button to initiate the request to SlideRule.  Depending on the size of your processing request, the response should take anywhere from 15 to 300 seconds.  Once all granules have been processed, the above map will be updated with a scatter plot of the different elevations.

In [ ]:
# callback for handling results
def demoCB(resource, result, index, total):
    global parms, results
    print(f'SlideRule processing request... [{index}/{total}] {resource} returned {len(results)} elevations\r', end="")
    results.append(result)

# build and transmit requests to SlideRule
def runSlideRule():
    global url_textbox, parms, results
    
    # set the url for the sliderule service
    if url_textbox.value == 'local':
        url = 'host.docker.internal'
    else:
        url = url_textbox.value
    icesat2.init(url, loglevel=logging.WARNING)

    # sliderule asset and data release
    asset = SRwidgets.asset.value

    # build sliderule parameters using latest values from widget
    parms = {
        # surface type: 0-land, 1-ocean, 2-sea ice, 3-land ice, 4-inland water
        "srt": SRwidgets.surface_type.index,
        # length of ATL06-SR segment in meters
        "len": SRwidgets.length.value,
        # step distance for successive ATL06-SR segments in meters
        "res": SRwidgets.step.value,
        # confidence level for PE selection
        "cnf": SRwidgets.confidence.value,
        # ATL08 land surface classifications
        "atl08_class": list(SRwidgets.land_class.value),
        # maximum iterations, not including initial least-squares-fit selection
        "maxi": SRwidgets.iteration.value,
        # minimum along track spread
        "ats": SRwidgets.spread.value,
        # minimum PE count
        "cnt": SRwidgets.count.value,
        # minimum height of PE window in meters
        "H_min_win": SRwidgets.window.value,
        # maximum robust dispersion in meters
        "sigma_r_max": SRwidgets.sigma.value
    }

    # clear existing geodataframe
    results = []
    rsps = icesat2.__emptyframe()

    # for each region of interest
    for poly in m.regions:
        # add polygon from map to sliderule parameters
        parms["poly"] = poly 
        # make the request to the SlideRule (ATL06-SR) endpoint
        # and pass it the request parameters to request ATL06 Data
        icesat2.atl06p(parms, asset, callback=demoCB)

    rsps = geopandas.pd.concat(results)
    return rsps

# run sliderule action
def on_run_clicked(b):
    global atl06_rsps
    with run_output:
        print(f'SlideRule processing request... initiated\r', end="")
        perf_start = time.perf_counter()
        atl06_rsps = runSlideRule()
        perf_duration = time.perf_counter() - perf_start
        print(f'SlideRule processing request... completed in {perf_duration:.3f} seconds; returned {atl06_rsps.shape[0]} records                                   ')
        m.GeoData(atl06_rsps, column_name="h_mean", cmap="viridis")

# create button to run sliderule
run_button = widgets.Button(description="Run SlideRule!")
run_output = widgets.Output()
run_button.on_click(on_run_clicked)
display(run_output)
display(run_button)

In [ ]:
# url input text box
url_textbox = widgets.Text(
    value='local',
    placeholder='Input box for SlideRule url',
    description='URL:',
    disabled=False
)
display(url_textbox)

# display widgets for setting SlideRule parameters
widgets.VBox([
    SRwidgets.surface_type,
    SRwidgets.length,
    SRwidgets.step,
    SRwidgets.confidence,
    SRwidgets.land_class,
    SRwidgets.iteration,
    SRwidgets.spread,
    SRwidgets.count,
    SRwidgets.window,
    SRwidgets.sigma,
])

See http://icesat2sliderule.org/rtd/user_guide/ICESat-2.html#elevations for descriptions of each column

### Step 3: Explore Data Points and Refresh Plot

Choose a field in the returned results to plot.

In [ ]:
# refresh action
def on_refresh_clicked(b):
    global atl06_rsps
    with refresh_output:
        m.GeoData(atl06_rsps, column_name=SRwidgets.variable.value, cmap=SRwidgets.colormap)

# create button to display geodataframe        
refresh_button = widgets.Button(description="Refresh Plot")
refresh_output = widgets.Output()
refresh_button.on_click(on_refresh_clicked)
display(refresh_button, refresh_output)

In [ ]:
widgets.VBox([
    SRwidgets.variable,
    SRwidgets.cmap,
    SRwidgets.reverse
])

### Step 4: Plot Photon Cloud of Ground Track

Enter the reference ground track, the cycle, and the individual ground track (10: GT1L, 20: GT1R, 30: GT2L, 40: GT2R, 50: GT3L, 60: GT3R) in the input boxes below.  If you click on an individual elevation in the map, it will automatically populate these inputs with the correct values.

In [ ]:
# atl03 plotting imports
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# globals for atl03 processing
atl03_rsps = None
rgt_textbox = None
cycle_textbox = None
gt_textbox = None

# track lookup tables
gtlookup = {icesat2.GT1L: 1, icesat2.GT1R: 1, icesat2.GT2L: 2, icesat2.GT2R: 2, icesat2.GT3L: 3, icesat2.GT3R: 3}
pairlookup = {icesat2.GT1L: 0, icesat2.GT1R: 1, icesat2.GT2L: 0, icesat2.GT2R: 1, icesat2.GT3L: 0, icesat2.GT3R: 1}

In [ ]:
# ATL03 Subsetter
def runATL03Subsetter():
    global url_textbox, rgt_textbox, cycle_textbox, gt_textbox, parms
    
    # set the url for the sliderule service
    if url_textbox.value == 'local':
        url = 'host.docker.internal'
    else:
        url = url_textbox.value
    icesat2.init(url, loglevel=logging.WARNING)

    # sliderule asset and data release
    asset = SRwidgets.asset.value

    # build sliderule parameters using latest values from widget
    parms = {
        # surface type: 0-land, 1-ocean, 2-sea ice, 3-land ice, 4-inland water
        "srt": SRwidgets.surface_type.index,
        # length of ATL06-SR segment in meters
        "len": SRwidgets.length.value,
        # step distance for successive ATL06-SR segments in meters
        "res": SRwidgets.step.value,
        # confidence level for PE selection
        "cnf": SRwidgets.confidence.value,
        # ATL08 land surface classifications
        "atl08_class": list(SRwidgets.land_class.value),
        # minimum along track spread
        "ats": SRwidgets.spread.value,
        # minimum PE count
        "cnt": SRwidgets.count.value,
        
        # region of interest
        "poly": m.regions[0],
        
        # track selection
        "rgt": int(rgt_textbox.value),
        "cycle": int(cycle_textbox.value),
        "track": gtlookup[int(gt_textbox.value)]
    }

    # make call to sliderule
    rsps = icesat2.atl03sp(parms, asset)
    
    # return geodataframe
    return rsps

# photon_cloud action
def on_pc_clicked(b):
    global atl03_rsps
    with pc_output:
        print(f'SlideRule processing request... initiated\r', end="")
        perf_start = time.perf_counter()
        atl03_rsps = runATL03Subsetter()
        perf_duration = time.perf_counter() - perf_start
        print(f'SlideRule processing request... completed in {perf_duration:.3f} seconds; returned {atl03_rsps.shape[0]} records                                   ')
        # Create Plots
        fig = plt.figure(num=None, figsize=(24, 8))
        colormap = np.array(['c','b','g','g','y']) # noise, ground, canopy, top of canopy, unclassified
        ax = plt.subplot(111)
        ax.set_title("Photon Cloud")
        gt_rsps = atl03_rsps[atl03_rsps["pair"] == pairlookup[int(gt_textbox.value)]]
        plot = ax.scatter(gt_rsps.index.values, gt_rsps["height"].values, c=colormap[gt_rsps["atl08_class"]], s=1.5)
        display(plt.show())

# create button to display geodataframe        
pc_button = widgets.Button(description="Plot Photon Cloud")
pc_output = widgets.Output()
pc_button.on_click(on_pc_clicked)
display(pc_button, pc_output)

In [ ]:
# reference ground track
rgt_textbox = widgets.Text(
    value='0',
    placeholder='Input box for reference groud track',
    description='RGT:',
    disabled=False
)

# cycle input text box
cycle_textbox = widgets.Text(
    value='0',
    placeholder='Input box for cycle',
    description='Cycle:',
    disabled=False
)

# ground track input text box
gt_textbox = widgets.Text(
    value='0',
    placeholder='Input box for ground track',
    description='Track:',
    disabled=False
)

display(rgt_textbox)
display(cycle_textbox)
display(gt_textbox)


def click_handler(feature):
    if "properties" in feature:
        if "rgt" in feature["properties"] and "cycle" in feature["properties"] and "gt" in feature["properties"]:
            rgt_textbox.value = str(feature["properties"]["rgt"])
            cycle_textbox.value = str(feature["properties"]["cycle"])
            gt_textbox.value = str(feature["properties"]["gt"])
    
m.add_selected_callback(click_handler)